<a href="https://colab.research.google.com/github/Birkbeck/bsc-computer-science-project-2021_22-mohammadreza490/blob/main/project_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
'''

1. Convert stems to wav. https://pypi.org/project/musdb/ 
2. For each song, split into 2 second segments. Add padding as needed
1. For each segment, create spectogram of whole mix of each song in training set via STFT
2. Create array containing spectograms of bass of each song in training set
3. Create array containing spectograms of (whole mix - bass) of each song in training set, see https://stackoverflow.com/questions/4039158/mixing-two-audio-files-together-with-python
4. Add padding to arrays such that shape is appropriate for the network.
4. Create CNN model
5. Train model
6. Apply inverse STFT / Get signal from the spectogram, see https://stackoverflow.com/questions/60377585/how-can-i-reverse-a-scipy-signal-spectrogram-to-audio-with-python
                                                        https://stackoverflow.com/questions/76447360/how-do-i-can-reconstructing-stft-to-audio
                                                        https://stackoverflow.com/questions/69387104/how-to-convert-wav-audio-file-from-mel-spectrogram
7. Test model with a bunch of songs

https://medium.com/@shameerayaseen21/u-net-advancing-image-segmentation-with-convolutional-neural-networks-1fd810f05d00
'''

'\n\n1. Convert stems to wav. https://pypi.org/project/musdb/ \n2. For each song, split into 2 second segments. Add padding as needed\n1. For each segment, create spectogram of whole mix of each song in training set via STFT\n2. Create array containing spectograms of bass of each song in training set\n3. Create array containing spectograms of (whole mix - bass) of each song in training set, see https://stackoverflow.com/questions/4039158/mixing-two-audio-files-together-with-python\n4. Add padding to arrays such that shape is appropriate for the network.\n4. Create CNN model\n5. Train model\n6. Apply inverse STFT / Get signal from the spectogram, see https://stackoverflow.com/questions/60377585/how-can-i-reverse-a-scipy-signal-spectrogram-to-audio-with-python\n                                                        https://stackoverflow.com/questions/76447360/how-do-i-can-reconstructing-stft-to-audio\n                                                        https://stackoverflow.com/ques

In [ ]:
PATH_TO_SRC_FOLDER = r".\src"

In [ ]:
!pip install musdb
!pip install numpy
!pip install pydub
!pip install tensorflow==2.14
!pip install tensorflow-addons==0.22.0
!pip install ipdb
!pip install museval
!pip install librosa
!pip install matplotlib


In [1]:
import os
import sys
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import IPython.display as ipd
import musdb
import subprocess
import pydub
import tensorflow_addons as tfa
import time
import shutil
import ipdb
import scipy as sp
import soundfile as sf
import museval
import random
import math

from scipy.io.wavfile import write
from glob import glob

C:\Users\Rohan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
import sys

if PATH_TO_SRC_FOLDER not in sys.path:
  sys.path.append(PATH_TO_SRC_FOLDER) #https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab

In [4]:
from wav_handler import WavHandler

In [9]:
y, sr = librosa.load(r"dataset\wav\test\Al James - Schoolboy Facination\mixture.wav", sr=None)

stft = librosa.core.stft(y, n_fft=2048, hop_length=512)
spectrogram = np.abs(stft)
mel_spectrogram = librosa.amplitude_to_db(spectrogram)

print(mel_spectrogram.shape)

(1025, 17257)


In [11]:
wav_handler = WavHandler(r"dataset\wav\test\Al James - Schoolboy Facination\mixture.wav")

segmented = wav_handler.segmentWav()
spectrograms = wav_handler.computeSTFT()
mel_spectrograms = wav_handler.computeMelSpec()

istft = wav_handler.computeInverseSTFT(spectrograms)

print(istft.shape)
print(wav_handler.wav.shape)

write("reconstructed.wav", rate = wav_handler.sr, data = istft)

Reconstructed spectrogram shape:  (1025, 17473)
Orig spectrogram shape:  (1025, 17473)
Phases array shape:  (1025, 17473)
Complex valued spec:  (1025, 17473)
(8945664,)
(8835072,)


In [31]:
PATH_TO_MUSDB_TRAIN = r".\dataset\wav\train\*\*.wav"
PATH_TO_MUSDB_TEST = r".\dataset\wav\test\*\*.wav"

x = PATH_TO_MUSDB_TRAIN.split("\\")

train_stems = glob(PATH_TO_MUSDB_TRAIN)
test_stems = glob(PATH_TO_MUSDB_TEST)

print(train_stems)
print(x)

print(glob(r".\dataset\wav\train\*")[1])

y, sr = librosa.load(glob(r".\dataset\wav\train\*")[1] + "\\bass.wav", sr = None)

print(len(y))

['.\\dataset\\wav\\train\\A Classic Education - NightOwl\\accompaniment.wav', '.\\dataset\\wav\\train\\A Classic Education - NightOwl\\bass.wav', '.\\dataset\\wav\\train\\A Classic Education - NightOwl\\drums.wav', '.\\dataset\\wav\\train\\A Classic Education - NightOwl\\linear_mixture.wav', '.\\dataset\\wav\\train\\A Classic Education - NightOwl\\mixture.wav', '.\\dataset\\wav\\train\\A Classic Education - NightOwl\\other.wav', '.\\dataset\\wav\\train\\A Classic Education - NightOwl\\vocals.wav', '.\\dataset\\wav\\train\\Actions - Devil_s Words\\accompaniment.wav', '.\\dataset\\wav\\train\\Actions - Devil_s Words\\bass.wav', '.\\dataset\\wav\\train\\Actions - Devil_s Words\\drums.wav', '.\\dataset\\wav\\train\\Actions - Devil_s Words\\linear_mixture.wav', '.\\dataset\\wav\\train\\Actions - Devil_s Words\\mixture.wav', '.\\dataset\\wav\\train\\Actions - Devil_s Words\\other.wav', '.\\dataset\\wav\\train\\Actions - Devil_s Words\\vocals.wav', '.\\dataset\\wav\\train\\Actions - One Minut

## THE FOLLOWING STUFF BELOW ARE 'SCRIBBLES'

In [ ]:
# !musdbconvert "C:\Users\Rohan\Desktop\DSP\Final-Year-Project\Dataset\musdb18" "C:\Users\Rohan\Desktop\DSP\Final-Year-Project\Dataset\musdb18wav"

In [ ]:
PATH_TO_MUSDB_TRAIN = r"dataset\wav\train\*"
PATH_TO_MUSDB_TEST = r"dataset\wav\test\*"

PATH_TO_TRAIN_SONGS = glob(PATH_TO_MUSDB_TRAIN)
PATH_TO_TEST_SONGS = glob(PATH_TO_MUSDB_TEST)

for song in PATH_TO_TRAIN_SONGS:
    print(glob(song + "\*.wav"))

['dataset\\wav\\train\\A Classic Education - NightOwl\\accompaniment.wav', 'dataset\\wav\\train\\A Classic Education - NightOwl\\bass.wav', 'dataset\\wav\\train\\A Classic Education - NightOwl\\drums.wav', 'dataset\\wav\\train\\A Classic Education - NightOwl\\linear_mixture.wav', 'dataset\\wav\\train\\A Classic Education - NightOwl\\mixture.wav', 'dataset\\wav\\train\\A Classic Education - NightOwl\\other.wav', 'dataset\\wav\\train\\A Classic Education - NightOwl\\vocals.wav']
['dataset\\wav\\train\\Actions - Devil_s Words\\accompaniment.wav', 'dataset\\wav\\train\\Actions - Devil_s Words\\bass.wav', 'dataset\\wav\\train\\Actions - Devil_s Words\\drums.wav', 'dataset\\wav\\train\\Actions - Devil_s Words\\linear_mixture.wav', 'dataset\\wav\\train\\Actions - Devil_s Words\\mixture.wav', 'dataset\\wav\\train\\Actions - Devil_s Words\\other.wav', 'dataset\\wav\\train\\Actions - Devil_s Words\\vocals.wav']
['dataset\\wav\\train\\Actions - One Minute Smile\\accompaniment.wav', 'dataset\\wav\

In [ ]:
# SPLIT SONG INTO 2-SECOND SEGMENTS
# TRANSFORM IT SO THAT THERE IS NO REMAINDER (ALL SEGMENTS ARE 2 SECONDS)

In [ ]:
# NOTE: librosa is built on top of matplotlib

# TIME DOMAIN

In [ ]:
# FREQUENCY DOMAIN (FFT) -> 'Snapshot' of the whole range and the average magnitudes throughout the signal

In [ ]:
# TIME-FREQUENCY DOMAIN (STFT) -> Computes the transforms of several windows throughout the signal, with the color representing the magnitude (similar to a heat map)

# COMPUTE STFT

# Shape: (1025, 188) for 48000 kHz
# (1025, 173) for 44100 kHz

In [ ]:
# FROM SPECTOGRAM, COMPUTE ISTFT

# step2 - converting audio np array to spectrogram
spec = librosa.feature.melspectrogram(y=split[SEGMENT_INDEX],
                                        sr=sr, 
                                            n_fft=2048, 
                                            hop_length=512, 
                                            win_length=None, 
                                            window='hann', 
                                            center=True, 
                                            pad_mode='reflect', 
                                            power=2.0,
                                     n_mels=128)

# step3 converting mel-spectrogram back to wav file
res = librosa.feature.inverse.mel_to_audio(spec, 
                                           sr=sr, 
                                           n_fft=2048, 
                                           hop_length=512, 
                                           win_length=None, 
                                           window='hann', 
                                           center=True, 
                                           pad_mode='reflect', 
                                           power=2.0, 
                                           n_iter=32)

write('istft.wav', sr, res, axis=None)

NameError: name 'split' is not defined

In [ ]:
# CREATE FUNCTION THAT REPRODUCES THE SONG BUT WITHOUT THE BASS STEM

mus = musdb.DB(root=r"C:\Users\Rohan\Desktop\DSP\Final-Year-Project\Dataset\musdb18")

#mus_train = musdb.DB(subsets="train")
#mus_test = musdb.DB(subsets="test")          

for track in mus:
    print(track.name)
               

drums, sr = librosa.load(r"Dataset/musdb18wav/test/Al James - Schoolboy Facination/drums.wav", sr=None)
vocals, sr = librosa.load(r"Dataset/musdb18wav/test/Al James - Schoolboy Facination/vocals.wav", sr=None)
other, sr = librosa.load(r"Dataset/musdb18wav/test/Al James - Schoolboy Facination/other.wav", sr=None)

librosa.display.waveshow((drums + vocals + other)/3, sr=sr)

write('bassless.wav', sr, (drums + vocals + other)/3)

A Classic Education - NightOwl
ANiMAL - Clinic A
ANiMAL - Easy Tiger
ANiMAL - Rockshow
Actions - Devil_s Words
Actions - One Minute Smile
Actions - South Of The Water
Aimee Norwich - Child
Alexander Ross - Goodbye Bolero
Alexander Ross - Velvet Curtain
Angela Thomas Wade - Milk Cow Blues
Atlantis Bound - It Was My Fault For Waiting
Auctioneer - Our Future Faces
AvaLuna - Waterduct
BigTroubles - Phantom
Bill Chudziak - Children Of No-one
Black Bloc - If You Want Success
Celestial Shore - Die For Us
Chris Durban - Celebrate
Clara Berry And Wooldog - Air Traffic
Clara Berry And Wooldog - Stella
Clara Berry And Wooldog - Waltz For My Victims
Cnoc An Tursa - Bannockburn
Creepoid - OldTree
Dark Ride - Burning Bridges
Dreamers Of The Ghetto - Heavy Love
Drumtracks - Ghost Bitch
Faces On Film - Waiting For Ga
Fergessen - Back From The Start
Fergessen - Nos Palpitants
Fergessen - The Wind
Flags - 54
Giselle - Moss
Grants - PunchDrunk
Helado Negro - Mitad Del Mundo
Hezekiah Jones - Borrowed Hear

C:\Users\Rohan\AppData\Local\Temp\ipykernel_21828\4279261374.py:12: UserWarning: PySoundFile failed. Trying audioread instead.
  drums, sr = librosa.load(r"Dataset/musdb18wav/test/Al James - Schoolboy Facination/drums.wav", sr=None)
C:\Users\Rohan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'Dataset/musdb18wav/test/Al James - Schoolboy Facination/drums.wav'

In [ ]:
# CREATE U-NET MODEL

# Spectogram dimensions
IMG_WIDTH = 1040 # Padded
IMG_HEIGHT = 192 # Padded
IMG_CHANNELS = 1


# Build the model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs) # Normalize image values

# Contraction path -> Extract details
'''
relu ensures that values are nonnegative
kernel_initializer initializes weights(?) based on some probability distribution / function?
padding denotes that the dimensions should be the same

skip connections allow the model to "remember" what it's trying to recreate instead of just working with its details (i.e. features) https://ai.stackexchange.com/questions/37321/what-is-the-role-of-skip-connections-in-u-net
the dropout drops out nodes in the input and hidden layer with probability p to ensure that the model is not overfitting


C1:
16 filters that are 3 by 3, filters define the features
Drop 10%(?)
Another 16 filters that are 3 by 3
Max pooling layer 2 by 2 with stride 2 compresses the image to half its size

C2: 
32 filters that are 3 by 3
Drop 10%(?)
Another 32 filters
Max pooling layer halves the size again

C3:
...

C4:
...

C5:
...

'''
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

# Expansive path -> Upsample, recreate image
'''
C6:
Upsample C5 (i.e. Double the dimensions and halve the features) via transpose, call this U6
Apply skip connections via concatenate (Append features from C4 to U6)
Layer with half the features as C6 (Why?)

C7:
Upsample C6 (i.e. Double the dimensions and halve the features) via transpose, call this U7
Apply skip connections via concatenate (Append features from C3 to U7)
Layer with half the features as C6

C8:
...

C9:
...

Dense Layer:
Flatten the 128*128*16 and map (fully) to 128*128*1 pixels (original image) 

'''
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 192, 1040, 1)]       0         []                            
                                                                                                  
 lambda_1 (Lambda)           (None, 192, 1040, 1)         0         ['input_2[0][0]']             
                                                                                                  
 conv2d_29 (Conv2D)          (None, 192, 1040, 32)        320       ['lambda_1[0][0]']            
                                                                                                  
 dropout_14 (Dropout)        (None, 192, 1040, 32)        0         ['conv2d_29[0][0]']           
                                                                                            